In [ ]:
import re
import string
import urllib
from datetime import datetime
from zipfile import ZipFile

import benchlingapi
import Bio.pairwise2 as pairwise2
import Bio.Restriction as Restriction
import holoviews as hv
import hvplot.pandas
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pygsheets
import requests
import seaborn as sns
import toml
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from tqdm.auto import tqdm

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.api.benchling as bapi
import paulssonlab.cloning.enzyme as enzyme
import paulssonlab.cloning.registry as registry
import paulssonlab.cloning.sequence as sequence
import paulssonlab.cloning.workflow as workflow
from paulssonlab.api.util import base_url

In [ ]:
hv.extension("bokeh")

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
bench_session = benchlingapi.Session(config["benchling"]["api_key"])
benchling_folder = bapi.get_project_root(bench_session, config["benchling"]["project"])

In [ ]:
reg = registry.Registry(gc, config["registry"]["folder"], benchling_folder)

# Data

## 2013 Chen

In [ ]:
!mkdir -p data/2013terminators_supp
!curl -b does_not_exist -Lo data/2013terminators_supp/supptable2.xlsx "https://static-content.springer.com/esm/art%3A10.1038%2Fnmeth.2515/MediaObjects/41592_2013_BFnmeth2515_MOESM206_ESM.xlsx"
!curl -b does_not_exist -Lo data/2013terminators_supp/supptable3.xlsx "https://static-content.springer.com/esm/art%3A10.1038%2Fnmeth.2515/MediaObjects/41592_2013_BFnmeth2515_MOESM207_ESM.xlsx"

In [ ]:
natural_terminators = pd.read_excel(
    "data/2013terminators_supp/supptable2.xlsx", index_col=0
)
synthetic_terminators = pd.read_excel(
    "data/2013terminators_supp/supptable3.xlsx", index_col=0
)
for terminators in (natural_terminators, synthetic_terminators):
    terminators["Length"] = terminators["Sequence"].str.len()

## 2020 Park

In [ ]:
# FROM: used https://www.adobe.com/acrobat/online/pdf-to-excel.html to convert
# Table S1 (p. 23) from https://www.embopress.org/action/downloadSupplement?doi=10.15252%2Fmsb.20209584&file=msb209584-sup-0001-AppendixFig.pdf

voigt_bidirectional_terms_tsv = """
name	sequence	Ts_forward	Ts_reverse
DT3	"CCGGCTTATCGGTCAGTTTCACCTGATTTACGTAAAAACCCGCTTCGGCGGGTTTTTGCTTTTGGAGGGGCAGAAAGATGAATGACTGTCCACGACGCTATACCCAAAAGAAAAAAAAAAAACCCCGCCCCTGACAGGGCGGGGTTTTTTTT"		3000				120		
DT5	"TCCGGCAATTAAAAAAGCGGCTAACCACGCCGCTTTTTTTACGTCTGCACTCGGTACCAAATTCCAGAAAAGAGGCCTCCCGAAAGGGGGGCCTTTTTTCGTTTTGGTCC"		4700				50		
DT19	TTCAGCCAAAAAACTTAAGACCGCCGGTCTTGTCCACTACCTTGCAGTAATGCGGTGGACAGGATCGGCGGTTTTCTTTTCTCTTCTCAACTCGGTACCAAAGACGAACAATAAGACGCTGAAAAGCGTCTTTTTTCGTTTTGGTCC	770				1.2			
DT34	GCTGATGCCAGAAAGGGTCCTGAATTTCAGGGCCCTTTTTTTACATGGATTGCTCGGTACCAAATTCCAGAAAAGAGACGCTTTCGAGCGTCTTTTTTCGTTTTGGTCC	570				1.4			
DT36	GATCTAACTAAAAAGGCCGCTCTGCGGCCTTTTTTCTTTTCACTGTAACAACGGAAACCGGCCATTGCGCCGGTTTTTTTTGGCCT	680				3.2			
DT42	"AGTTAACCAAAAAGGGGGGATTTTATCTCCCCTTTAATTTTTCCTCGCAGATAGCAAAAAAGCGCCTTTAGGGCGCTTTTTTACATTG
GTGG"	2500				2.2			
DT54	"GGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGGCTCGGTACCAAATTCCAGAAAAGACACCCGAAAGGGTGTTTTTTCGTTTTGGTCC"		1800				30		
DT56	TACCACCGTCAAAAAAAACGGCGCTTTTTAGCGCCGTTTTTATTTTTCAACCTTCCAGGCATCAAATAAAACGAAAGGCTCAGTCGAAAGACTGGGCCTTTCGTTTTATCTGTTGTTTGTCGGTGAACGCTCTC	240				11			
DT60	ACATTTAATAAAAAAAGGGCGGTCGCAAGATCGCCCTTTTTTACGTATGACACAGTGAAAAATGGCGCCCATCGGCGCCATTTTTTTATG	110				29			
DT65	TGCTCGTACCAGGCCCCTGCAATTTCAACAGGGGCCTTTTTTTATCCAATTCCATCGGGTCCGAATTTTCGGACCTTTTCTCCGC	400				1.0			
DT82	"CTTATTCCATAACAAAGCCGGGTAATTCCCGGCTTTGTTGTATCTGAACAATAAATGGATGCCCTGCGTAAGCGGGGCATTTTTCTTCCT"	170				2.8			
DT83	AGCGTCAAAAGGCCGGATTTTCCGGCCTTTTTTATTAGGCAGCATGCTGCCAGGTGATCCCCCTGGCCACCTCTTTT	600				4.4			
DT86	TAATCATTCTTAGCGTGACCGGGAAGTCGGTCACGCTACCTCTTCTGAAGAAACAGCAAACAATCCAAAACGCCGCGTTCAGCGGCGTTTTTTCTGCTTTTCT	210				0.4			
DT100	"GTGAAGTGAAAAATGGCGCACATTGTGCGCCATTTTTTTTGTCTGCCGTTTACCGCTTCTCTGAAAATCAACGGGCAGGTCACTGACTTGCCCGTTTTTTTATCCCTTCTCCACACCG"	4700				12			
DT101	"TCTTTAAAAAGAAACCTCCGCATTGCGGAGGTTTCGCCTTTTGATACTCTGTCTGAAGTAATTCTTGCCGCAGTGAAAAATGGCGCCCATCGGCGCCATTTTTTTATGCTTCCATTAGAAAGCAAAAAGCCTGCTAGAAAGCAGGCTTTTTTGAATTTGGCTCCTCTGAC"		2800				160		
DT103	"AAAGTTCTGAAAAAGGGTCACTTCGGTGGCCCTTTTTTATCGCCACGGTTTGAGCAGTGCACTTGCTTAAAATCCCGCCAGCGGCGGGATTTTTTATTGTCCGGTTTAAGACA"	790				4.0			
DT104	"GCAGACAAAAAAAATGGCGCACAATGTGCGCCATTTTTCACTTCACAGGTACTATTGTTTTGAATTGAAAAGGGCGCTTCGGCGCCCTTTTTGCATTTGTTGACGGCATATATTTGTATATCGAAGCGCCCTGATGGGCGCTTTTTTTATTTAATCGATAACCAGA"		580				101		
"""

import io

voigt_bidirectional_terms = pd.read_csv(
    io.StringIO(voigt_bidirectional_terms_tsv), sep="\s+", index_col=0
)

## 2019 Hudson

In [ ]:
!mkdir -p data/2019hudson_supp
!curl -b does_not_exist -Lo data/2019hudson_supp.zip "https://oup.silverchair-cdn.com/oup/backfile/Content_public/Journal/synbio/4/1/10.1093_synbio_ysz026/2/ysz026_supplementary_data.zip?Expires=1625620668&Signature=ce7~qRkVxcwAzjs98YDSNU03XBjhxBoyL6Mr2RsQEordmOg8N8Fh5u0trhiAxYSZtzbF~U~x2DjUG5dEYLjtMiF-YTejDIOMqtPdmVd6-n4WK5wn02j8JS3whJt57SL6fCAYAlcBiVKPyPH-tTyctP84SK6v8~zjS07hmob6LY-MmBt-XDgaTMiks6Pqkw3yUfnaUZP4IbXnO0Nt~p5uFlE~iStyoaTz~Y4uGvLSjcA2YUmiCvUVQOlNVcXwPMpHkot6CnD0~ZhKMxE8w4dYhGs4DJk2xsrecIfKHS1WCWzyd6N20lAB43hHQNQEKr61oNSazYD9rCRfGIckt5Uepg__&Key-Pair-Id=APKAIE5G5CRDK6RD3PGA"

In [ ]:
with ZipFile("data/2019hudson_supp.zip") as zip:
    with zip.open(
        "Supplemental Table 3A - TermSeq FACS Binning Data_allover200reads.csv"
    ) as f:
        hudson_terminators = pd.read_csv(
            f, header=[2], thousands=",", na_values=["#DIV/0!"]
        )
hudson_terminators.loc[pd.isnull(hudson_terminators["Ave TS"]), "Ave TS"] = np.inf
for col in ("Ave", "SD", "CI (95% CL)"):
    hudson_terminators[col] = (
        pd.to_numeric(hudson_terminators[col].str.rstrip("%")) / 100
    )
hudson_terminators.rename(
    {"Terminator Sequence (5' to 3')": "Sequence"}, axis=1, inplace=True
)
hudson_terminators["Sequence_rev"] = hudson_terminators["Sequence"].map(
    lambda x: str(sequence.reverse_complement(x))
)
hudson_terminators = hudson_terminators.join(
    hudson_terminators[
        ["Sequence", "Ave TS", "Ave", "SD", "SD/AVE", "Term ID"]
    ].set_index("Sequence"),
    on="Sequence_rev",
    rsuffix="_rev",
)
hudson_terminators["Ave TS_min"] = hudson_terminators[["Ave TS", "Ave TS_rev"]].min(
    axis=1
)
hudson_terminators["Ave_min"] = hudson_terminators[["Ave", "Ave_rev"]].min(axis=1)
hudson_terminators = hudson_terminators[
    pd.isnull(hudson_terminators["Ave_rev"])
    | (hudson_terminators["Ave"] >= hudson_terminators["Ave_rev"])
    | (
        (np.isinf(hudson_terminators["Ave"]))
        & (hudson_terminators["Term ID"] >= hudson_terminators["Term ID_rev"])
    )
].copy()
hudson_terminators.set_index("Term ID", inplace=True)

# Functions

In [ ]:
from itertools import chain


def align(a, b):
    a = a.lower()
    b = b.lower()
    # return pairwise2.align.localxs(a, b, -0.5, -0.5)
    return pairwise2.align.localms(a, b, 1, -0.5, -0.5, -0.5)


def align_score(a, b):
    alignments = align(a, b)
    if not len(alignments):
        return 0
    else:
        return alignments[0].score


def show_alignment(a, b):
    print(pairwise2.format_alignment(*align(a, b)[0], full_sequences=True))


def homology(a, b):
    b_rc = str(sequence.reverse_complement(b))
    forward = align_score(a, b)
    reverse = align_score(a, b_rc)
    return max(forward, reverse)


def min_length(a, b):
    return min(len(a), len(b))


def similarity(a, b):
    return homology(a, b) / min_length(a, b)


def map_pairwise(func, xs, ys):
    mat = np.zeros((len(xs), len(ys)))
    if xs.equals(ys):
        for i in range(len(xs)):
            for j in range(i):
                mat[i, j] = mat[j, i] = func(xs[i], ys[j])
    else:
        for i in range(len(xs)):
            for j in range(len(ys)):
                mat[i, j] = func(xs[i], ys[j])
    return pd.DataFrame(mat, index=xs.index, columns=ys.index)


]def show_heatmap(df):
    with pd.option_context(
        "display.max_rows", None, "display.max_columns", None
    ):  # more options can be specified also
        display(df.style.background_gradient(cmap="RdPu", axis=None))


def get_terminator(terminators, names):
    seq = ""
    for name in names:
        if name.endswith("_r"):
            seq += str(sequence.reverse_complement(terminators.loc[name[:-2]]))
        else:
            seq += terminators.loc[name]
    return seq


def get_nonrepetitive(
    similarity_func,
    max_similarity,
    candidates,
    others,
    num,
    enzymes=[],
    blacklist=[],
):
    accepted = []
    accepted_keys = []
    idx = 0
    for i in range(num):
        while idx < len(candidates):
            if candidates.index[idx] in blacklist:
                idx += 1
                continue
            candidate = candidates.iloc[idx]
            if any(enzyme.re_search(candidate, enz) for enz in enzymes):
                idx += 1
                continue
            if not len(accepted) and not len(others):
                similarity = 0
            else:
                similarity = max(
                    similarity_func(candidate, seq) for seq in chain(accepted, others)
                )
            if similarity <= max_similarity:
                accepted.append(candidate)
                accepted_keys.append(candidates.index[idx])
                break
            idx += 1
        else:
            raise ValueError(f"ran out of candidates after accepting {i}")
    series = pd.Series(accepted, name=candidates.name, index=accepted_keys)
    series.index.name = candidates.index.name
    return series


def assemble_terminators(spec, parts):
    parts_per_assembly = sum(s[0] for s in spec)
    num_assemblies = len(parts) // parts_per_assembly
    assemblies = []
    for i in range(num_assemblies):
        assembly = list(
            parts[i : i + num_assemblies * parts_per_assembly : num_assemblies]
        )
        idx = 0
        for count, is_reversed in spec:
            for _ in range(count):
                if is_reversed:
                    assembly[idx] = f"{assembly[idx]}_r"
                idx += 1
        assemblies.append(assembly)
    return assemblies

# Voigt bidirectional terms

In [ ]:
selected_bidirectional_terms = voigt_bidirectional_terms[
    voigt_bidirectional_terms["Ts_reverse"] >= 10
]

In [ ]:
selected_bidirectional_terms

# Hudson bidirectional terms

In [ ]:
hudson_bidirectional = hudson_terminators[
    ~pd.isnull(hudson_terminators["Ave TS_rev"])
].copy()
hudson_bidirectional.sort_values("Ave_min", ascending=False, inplace=True)

In [ ]:
hudson_bidirectional

In [ ]:
hudson_bidirectional.plot.scatter("Ave TS", "Ave TS_rev")

In [ ]:
hudson_bidirectional[hudson_bidirectional["Ave_min"] > 0.995].plot.scatter(
    "Ave", "Ave_rev"
)

In [ ]:
plt.scatter(hudson_bidirectional["Ave TS"], np.log10(1 - hudson_bidirectional["Ave"]))

In [ ]:
%%time
hudson_sim = map_pairwise(
    similarity, hudson_bidirectional["Sequence"], hudson_bidirectional["Sequence"]
)

In [ ]:
%%time
selected_hudson = get_nonrepetitive(
    similarity, 0.7, hudson_bidirectional["Sequence"], [], 2
)

In [ ]:
selected_hudson

In [ ]:
hudson_bidirectional.loc[["T799", "T97", "T540"]]

In [ ]:
olib = reg[("oLIB", "oligos")]
olt = reg[("oLT", "oligos")]

In [ ]:
backbone_flanks = {
    "upstream": (
        sequence.reverse_complement(olib["oLIB179"]["Sequence"]),
        olib["oLIB180"]["Sequence"],
    ),
    "downstream": (
        sequence.reverse_complement(olib["oLIB181"]["Sequence"]),
        olib["oLIB182"]["Sequence"],
    ),
}

backbone_insulation_terminators = {
    "upstream": "T97",
    "downstream": "T540",
}

In [ ]:
date = datetime.now().strftime("%-m/%-d/%Y")

base_row = {
    "Author": "Jacob Quinn Shenker",
    "Vendor": "IDT",
    "Type": "Primer",
    "Date": date,
    "Order date": date,
}

rows = []

for orientation, terminator in backbone_insulation_terminators.items():
    seq = hudson_bidirectional.loc[terminator, "Sequence"]
    if orientation == "upstream":
        seq = sequence.reverse_complement(seq)
    seq = workflow.add_flanks(seq, [backbone_flanks[orientation]])
    for antisense in (False, True):
        if antisense:
            oligo_seq = sequence.reverse_complement(seq)
        else:
            oligo_seq = seq
        oligo_seq = str(oligo_seq)
        sense_str = "antisense" if antisense else "sense"
        name = f"JUMP_{orientation}_{terminator}_{sense_str}"
        term = hudson_bidirectional.loc[terminator]
        description = "2019 Hudson terminator {sense_str} oligo.\nTerminator efficiency: {te:.4f} / {te_rev:.4f} (reverse)\nTerminator strength (TS): {ts:.0f} / {ts_rev:.0f} (reverse)".format(
            sense_str=sense_str,
            te=term["Ave"],
            te_rev=term["Ave_rev"],
            ts=term["Ave TS"],
            ts_rev=term["Ave TS_rev"],
        )
        row = {
            "Name": name,
            "Sequence": oligo_seq,
            "Description": description,
            **base_row,
        }
        rows.append(row)

for row in rows:
    olt[olt.next_id()] = row

In [ ]:
olt.save()

In [ ]:
rows

In [ ]:
print(rows[-1]["Description"])

In [ ]:
str(upstream_backbone_insulation)

In [ ]:
str(downstream_backbone_insulation)

In [ ]:
hudson_bidirectional[["Ave TS", "Ave TS_rev"]][:20]

In [ ]:
show_heatmap(hudson_sim)

# Two forward+two reversed terminator (2r2) backbone insulation

In [ ]:
plt.scatter(synthetic_terminators["Average Strength"], synthetic_terminators["Length"])
plt.scatter(natural_terminators["Average Strength"], natural_terminators["Length"])

In [ ]:
all_terminators = pd.concat((synthetic_terminators, natural_terminators))
strong_all = all_terminators.sort_values("Average Strength", ascending=False)[
    ["Average Strength", "Length", "Sequence"]
][:80]
strong_all[-20:]

In [ ]:
strong_all_with_dts = pd.concat(
    (strong_all["Sequence"], selected_bidirectional_terms["sequence"])
)

In [ ]:
%%time
strong_all_sim = map_pairwise(similarity, strong_all_with_dts, strong_all_with_dts)

In [ ]:
ENZYMES = (Restriction.BsaI, Restriction.BsmBI, Restriction.BbsI, Restriction.AarI)

In [ ]:
%%time
insulation_pool_6 = get_nonrepetitive(
    similarity,
    0.5,
    natural_terminators["Sequence"],
    selected_bidirectional_terms["sequence"],
    6,
    enzymes=ENZYMES,
)

In [ ]:
insulation_pool_6

In [ ]:
strong_all.loc["L3S3P00"]

In [ ]:
strong_all.loc["ECK120010799"]

In [ ]:
strong_all.loc["ECK120010858-R"]

In [ ]:
strong_all.loc["BBa_B0062-R"]

In [ ]:
strong_all.loc["ECK120010869"]

In [ ]:
%%time
insulation_pool_8 = get_nonrepetitive(
    similarity,
    0.7,
    natural_terminators["Sequence"],
    selected_bidirectional_terms["sequence"],
    8,
    enzymes=ENZYMES,
    blacklist=["BBa_B0010", "pheA-1"],  # IDT seems unable to make pheA-1 as a gBlock
)

In [ ]:
insulation_pool_8

In [ ]:
show_heatmap(strong_all_sim[["pheA-1", "ECK120015440", "ECK120010799", "ECK120010855"]])

In [ ]:
show_heatmap(
    strong_all_sim[["ECK120029600", "ECK120033736", "BBa_B0062-R", "ECK120010869"]]
)

In [ ]:
show_heatmap(strong_all_sim)

In [ ]:
natural_terminators["Sequence"]  # [:100]

In [ ]:
%%time
insulation_pool_4 = get_nonrepetitive(
    similarity,
    0.7,
    natural_terminators["Sequence"],
    selected_bidirectional_terms["sequence"],
    4,
    enzymes=ENZYMES,
    # BBa_B0010 is already a double-term
    # IDT seems unable to make pheA-1 as a gBlock
    # last four are from oLT75 (which was synthesized successfully)
    blacklist=[
        "BBa_B0010",
        "pheA-1",
        "ECK120029600",
        "ECK120033736",
        "BBa_B0062-R",
        "ECK120010869",
    ],
)

In [ ]:
insulation_pool_4

In [ ]:
strong_all.loc["ECK120017009"]

In [ ]:
backbone_insulation_2r2 = assemble_terminators(
    ((2, False), (2, True)), insulation_pool_8.index
)

# want (slightly) stronger terms downstream (to insulate repA in pSC101)

backbone_insulation_2r2 = {
    "upstream": backbone_insulation_2r2[1],
    "downstream": backbone_insulation_2r2[0],
}

In [ ]:
backbone_insulation_2r2

In [ ]:
# backbone_insulation_2r2 = {
#     #"upstream": ["ECK120029600", "ECK120033736", "ECK120010799_r", "BBa_B0062-R_r"],
#     "upstream": ["pheA-1", "ECK120029600", "ECK120015440", "ECK120010799_r"],
#     "downstream": ["ECK120029600", "ECK120010858-R", "ECK120015440","ECK120010799_r"],
# }

In [ ]:
olib = reg[("oLIB", "oligos")]
olt = reg[("oLT", "oligos")]

In [ ]:
olt.clear_cache()

In [ ]:
backbone_flanks = {
    "upstream": (
        sequence.reverse_complement(olib["oLIB179"]["Sequence"]),
        olib["oLIB180"]["Sequence"],
    ),
    "downstream": (
        sequence.reverse_complement(olib["oLIB181"]["Sequence"]),
        olib["oLIB182"]["Sequence"],
    ),
}

In [ ]:
date = datetime.now().strftime("%-m/%-d/%Y")

base_row = {
    "Author": "Jacob Quinn Shenker",
    "Vendor": "IDT",
    "Type": "IDT gBlock",
    "Date": date,
    "Order date": date,
}

rows = []


def strip(s, suffix):
    if s.endswith(suffix):
        return s[: -len(suffix)]
    else:
        return s


for orientation, terms in backbone_insulation_2r2.items():
    seq = get_terminator(natural_terminators["Sequence"], terms)
    if orientation == "upstream":
        seq = sequence.reverse_complement(seq)
    seq = workflow.add_flanks(seq, [backbone_flanks[orientation]])
    oligo_seq = str(seq)
    name = f"JUMP_{orientation}_2r2"
    description = "Two forward terminators concatenated with two reverse terminators (no spacers, all taken from 2013 Chen et al.).\nTerminators: {terms}\nTerminator strength (TS): {ts}".format(
        terms=" / ".join(terms),
        ts=" / ".join(
            [
                "{:.0f}".format(
                    natural_terminators.loc[strip(term, "_r"), "Average Strength"]
                )
                for term in terms
            ]
        ),
    )
    row = {
        "Name": name,
        "Sequence": oligo_seq,
        "Description": description,
        **base_row,
    }
    rows.append(row)

In [ ]:
for row in rows:
    olt[olt.next_id()] = row

In [ ]:
for enz in (Restriction.BsaI, Restriction.BsmBI, Restriction.BbsI, Restriction.AarI):
    for olt_num in range(70, 76):
        cuts = enzyme.re_search(olt[f"oLT{olt_num}"]["Sequence"], enz)
        if cuts:
            print("!!!", olt_num, enz, cuts)

In [ ]:
olt.save()

In [ ]:
rows

In [ ]:
print(rows[0]["Description"])

# Similarity

In [ ]:
show_alignment(selected_pool["L3S1P56"], selected_pool["L3S3P00"])

In [ ]:
show_alignment(selected_pool["L3S3P47"], selected_pool["L3S3P00"])

In [ ]:
show_alignment(selected_pool["L3S3P47"], selected_pool["L3S3P41"])

In [ ]:
%%time
show_heatmap(map_pairwise(similarity, selected_pool, selected_pool))

In [ ]:
%%time
show_heatmap(map_pairwise(similarity, selected_pool, selected_pool))

In [ ]:
show_heatmap(pool_sim)

In [ ]:
%%time
hmat = map_pairwise(
    homology, strong_synthetic["Sequence"], strong_synthetic["Sequence"]
)

In [ ]:
lengths = map_pairwise(
    min_length, synthetic_terminators["Sequence"], selected_terms["sequence"]
)
nhmat = hmat / lengths

In [ ]:
get_terminator(synthetic_terminators["Sequence"], [""])

In [ ]:
cm = sns.light_palette("orange", as_cmap=True)
with pd.option_context(
    "display.max_rows", None, "display.max_columns", None
):  # more options can be specified also
    display(nhmat.style.background_gradient(cmap=cm))

In [ ]:
a = synthetic_terminators.loc["L1U8H11"]["Sequence"]
b = selected_terms.loc["DT5"]["sequence"]
alignments = pairwise2.align.localxs(a, b, -0.5, -0.5)
print(pairwise2.format_alignment(*alignments[0]))

In [ ]:
a = synthetic_terminators.loc["L3S2P21"]["Sequence"]
b = selected_terms.loc["DT5"]["sequence"]
alignments = pairwise2.align.localxs(a, b, -0.5, -0.5)
print(pairwise2.format_alignment(*alignments[0], full_sequences=True))

In [ ]:
alignments = pairwise2.align.localxs(ts[0], ts[2], -0.5, -0.5)
print(pairwise2.format_alignment(*alignments[0]))

In [ ]:
alignments = pairwise2.align.localxs(
    ts[0], str(sequence.reverse_complement(ts[2])), -0.5, -0.5
)
print(pairwise2.format_alignment(*alignments[0]))